In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style

style.use(
    "https://raw.githubusercontent.com/dominik-strutz/dotfiles/main/mystyle.mplstyle"
)

In [ ]:
def cable_attenuation(distance, attenuation_rate=0.4):
    """
    Calculate signal amplitude based on distance with attenuation of 0.4 dB/kilometer.

    Parameters:
    -----------
    distance : float or numpy.ndarray
        Distance in kilometers

    Returns:
    --------
    float or numpy.ndarray
        Signal amplitude (not in dB)
    """
    # 0.4 dB/kilometer attenuation
    # For amplitude (not power), we divide by 20 in the dB conversion
    attenuation_rate = attenuation_rate / 20.0

    # Calculate relative amplitude using the dB formula
    relative_distance = distance
    amplitude = 10 ** (-attenuation_rate * relative_distance / 1e3)

    return amplitude

In [ ]:
from dased.helpers.srcloc import MagnitudeRelation

# Define the distance values
distance_values = np.linspace(0, 25000, 1000)
reference_distance = 20000.0


reference_relation = MagnitudeRelation(
    magnitude_factor=0.437,
    log_coeff=-1.269,
    reference_distance=reference_distance,
)

# Create instances of the decay classes
spherical_decay = MagnitudeRelation(
    magnitude_factor=0.437, log_coeff=-2, reference_relation=reference_relation
)(distance_values)
cylindrical_decay = MagnitudeRelation(
    magnitude_factor=0.437, log_coeff=-1, reference_relation=reference_relation
)(distance_values)
p_wave = MagnitudeRelation(
    magnitude_factor=0.437, log_coeff=-1.269, reference_relation=reference_relation
)(distance_values)
s_wave = MagnitudeRelation(
    magnitude_factor=0.69, log_coeff=-1.588, reference_relation=reference_relation
)(distance_values)

# Calculate inverse cable attenuation
att_standard = cable_attenuation(distance_values, attenuation_rate=0.2)
att_enhanced = cable_attenuation(distance_values, attenuation_rate=1.0)
# Westbrook 2020

# Create figure and axis objects for two subplots stacked vertically
fig, axs = plt.subplots(2, 1, figsize=(4, 6), dpi=150, gridspec_kw={"hspace": 0.4})

# --- Plot 1: Decay Curves ---
ax1 = axs[0]

# Add horizontal and vertical reference lines
ax1.axhline(y=1.0, color="black", linestyle="-", linewidth=1.0)
ax1.axvline(
    x=reference_distance / 1e3, color="k", linestyle="-", linewidth=1.0, alpha=1.0
)

# text about the reference distance
ax1.text(
    reference_distance / 1e3 + 0.5,
    1.05e3,
    "reference distance",
    fontsize=6,
    color="black",
    ha="center",
    va="bottom",
)

# Plot different decay curves
ax1.plot(
    distance_values / 1e3, p_wave, label="P-Wave Decay (Yin et al. 2023)", linewidth=2.0
)
ax1.plot(
    distance_values / 1e3,
    spherical_decay,
    label="Spherical Decay",
    color="gray",
    linewidth=2.0,
    linestyle="--",
)
ax1.plot(
    distance_values / 1e3,
    cylindrical_decay,
    label="Cylindrical Decay",
    color="gray",
    linewidth=2.0,
    linestyle=":",
)
ax1.plot(
    distance_values / 1e3,
    s_wave,
    label="S-Wave Decay (Yin et al. 2023)",
    linewidth=2.0,
    linestyle="--",
)


# Set grid, scale, limits, and labels for Plot 1
ax1.grid(True, which="both", ls="-", alpha=0.2)
ax1.set_xlabel("distance from source (km)")
ax1.set_yscale("log")
ax1.set_ylim(0.5, 1000)
ax1.set_ylabel("relative amplitude (log scale)")
ax1.legend(facecolor="white", loc="upper right", fontsize=7)

# --- Plot 2: Inverse Cable Attenuation ---
ax2 = axs[1]

# Add horizontal reference line
ax2.axhline(y=1.0, color="black", linestyle="-", linewidth=1.0)

# reset color cycle
ax2.set_prop_cycle(None)

# Plot inverse cable attenuation
ax2.plot(
    distance_values / 1e3,
    att_standard,
    label="standard cable (0.2 dB/km)",
    linestyle="-",
    linewidth=2.0,
)

ax2.plot(
    distance_values / 1e3,
    att_enhanced,
    label="enhanced cable (0.5 dB/km)",
    linewidth=2.0,
)


# Set grid, scale, limits, and labels for Plot 2
ax2.grid(True, which="both", ls="-", alpha=0.2)
ax2.set_yscale("log")
ax2.set_ylim(0.08, 1.2)  # Adjusted ylim for attenuation plot
ax2.set_xlabel("distance along the cable (km)")
ax2.set_ylabel("relative amplitude (log scale)")
ax2.legend(facecolor="white", loc="lower left", fontsize=7)

fig.savefig("figures/attenuation_decay.png", dpi=300)
fig.savefig("figures/attenuation_decay.pdf", dpi=300)

plt.show()